Traddif light classifier learning

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [3]:
#Load data list
PATH_IMG_LIST_CSV="./data_sim.csv"
PATH_IMG_DIR="."
#PATH_IMG_LIST_CSV="../ros/src/tl_detector/training_data/data.csv"
#PATH_IMG_DIR="../ros/src/tl_detector"
img_list_csv=pd.read_csv(PATH_IMG_LIST_CSV,sep=',',header=None)
cls_raw=img_list_csv[1]
filenames=img_list_csv[0]
data_num_raw=len(cls_raw)
data_num_raw

2918

In [21]:
#Parameters
batch_size=64
learning_rate=1e-3
num_epoch=800
resized_width=256
resized_height=192

In [5]:
#Load images
#resize 600,800->292,256

data_num=data_num_raw*2
data=np.zeros([data_num_raw*2,resized_width,resized_height,3])
cls=np.zeros(data_num_raw*2)
for i,fn in enumerate(filenames):
    image_bgr = cv2.imread(PATH_IMG_DIR+"/"+fn)
    #image=cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
    image=cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HLS)
    resized_image=cv2.resize(image,(resized_height,resized_width))
    data[i,:,:,:]=resized_image
    #Data Augmentation
    data[i+data_num_raw,:,:,:]=resized_image[::-1,:,:]
    cls[i]=cls_raw[i]
    cls[i+data_num_raw]=cls_raw[i]
data.shape[0]

5836

In [6]:
#Split data

#Shuffle data
np.random.seed(0)
shuffle_idx=np.random.permutation(cls.shape[0])

#Split data indices
train_num=int(data_num*0.6)
test_num=int(data_num*0.2)
val_num=data_num-train_num-test_num
train_idx=shuffle_idx[:train_num]
test_idx=shuffle_idx[train_num:train_num+test_num]
val_idx=shuffle_idx[train_num+test_num:]

#Save indices
np.savetxt("dataset_train.csv",train_idx)
np.savetxt("dataset_test.csv",test_idx)
np.savetxt("dataset_val.csv",val_idx)

#Split data
train_cls=cls[train_idx]
train_data=data[train_idx]
test_cls=cls[test_idx]
test_data=data[test_idx]
val_cls=cls[val_idx]
val_data=data[val_idx]

In [7]:
#Define model

#Input
x_input_shape=(None,resized_width,resized_height,3)
y_input_shape=(None,)
x_input=tf.placeholder(tf.float32,shape=x_input_shape,name="x_input")
y_input=tf.placeholder(tf.int32,shape=y_input_shape,name="y_input")
y_input_onehot=tf.one_hot(y_input, depth=3, dtype=tf.float32)

keep_prob=tf.placeholder(tf.float32,name="keep_prob")

#conv 1
#192,256 -> 192,256 *32
conv1_w=tf.Variable(tf.truncated_normal([3,3,3,32]))
conv1_b=tf.Variable(tf.truncated_normal([32]))

conv1_c=tf.nn.conv2d(x_input,conv1_w,strides=[1,1,1,1], padding='SAME')+conv1_b
conv1_a=tf.nn.relu(conv1_c)

#dropout 1
drop1 = tf.nn.dropout(conv1_a, keep_prob)

#pool 1
#192,256 *32 -> 96,128 *32
pool1=tf.nn.max_pool(drop1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


#conv 2
#96,128*32 -> 96,128*64
conv2_w=tf.Variable(tf.truncated_normal([3,3,32,64]))
conv2_b=tf.Variable(tf.truncated_normal([64]))

conv2_c=tf.nn.conv2d(pool1,conv2_w,strides=[1,1,1,1], padding='SAME')+conv2_b
conv2_a=tf.nn.relu(conv2_c)

#dropout 2
drop2 = tf.nn.dropout(conv2_a, keep_prob)

#pool 2
#96,128*64 -> 48*64*64
pool2=tf.nn.max_pool(drop2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#conv 3
#48*64*64 -> 48*64*128
conv3_w=tf.Variable(tf.truncated_normal([3,3,64,128]))
conv3_b=tf.Variable(tf.truncated_normal([128]))

conv3_c=tf.nn.conv2d(pool2,conv3_w,strides=[1,1,1,1], padding='SAME')+conv3_b
conv3_a=tf.nn.relu(conv3_c)

#pool 3
#48*64*128 -> 24*32*128
pool3=tf.nn.max_pool(conv3_a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#fc
#24*32*128 ->3
fc1_w=tf.Variable(tf.truncated_normal([24*32*128,3]))
fc1_b=tf.Variable(tf.truncated_normal([3]))

fc1_in=tf.reshape(pool3,[-1,24*32*128])
fc1_c=tf.matmul(fc1_in,fc1_w)+fc1_b

#Prediction, accuracy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_input_onehot, logits=fc1_c))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
prediction=tf.argmax(fc1_c, 1,name="prediction")
correct_prediction = tf.equal(prediction, tf.argmax(y_input_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")

In [8]:
def eval_acc(x,y,batch_size):
    test_pred=np.zeros(0)
    for i in range(int(x.shape[0]/batch_size)):
        start_idx=i*batch_size
        end_idx=min((i+1)*batch_size,data.shape[0]-1)
        cur_x=x[start_idx:end_idx,:,:,:]
        cur_y=y[start_idx:end_idx]
        tmp=correct_prediction.eval(feed_dict={x_input: cur_x, y_input: cur_y, keep_prob: 1.0})
        test_pred=np.concatenate([test_pred,tmp],axis=0)
    test_accuracy=np.mean(test_pred)
    return test_accuracy

In [24]:
#Model test
tf.set_random_seed(0)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(num_epoch):
        for i in range(int(train_data.shape[0]/batch_size)):
            start_idx=i*batch_size
            end_idx=min((i+1)*batch_size,data.shape[0]-1)
            cur_x=train_data[start_idx:end_idx,:,:,:]
            cur_y=train_cls[start_idx:end_idx]
            train_step.run(feed_dict={x_input: cur_x, y_input: cur_y, keep_prob: 0.7})
        if j % 20==0:
            train_accuracy=accuracy.eval(feed_dict={x_input: cur_x, y_input: cur_y, keep_prob: 1.0})
            #test_accuracy=accuracy.eval(feed_dict={x_input: test_data, y_input: test_cls, keep_prob: 1})
            test_accuracy=eval_acc(test_data,test_cls,batch_size)
            print('epoch %d, train acc %f  test acc %f' % (j, train_accuracy,test_accuracy))
            #Save model
            save_path = saver.save(sess, "./tl_classifier.ckpt")
    valid_acc=eval_acc(test_data,val_cls,batch_size)
    print('validation acc %g' % valid_acc)

epoch 0, train acc 0.375000  test acc 0.381944
epoch 20, train acc 0.531250  test acc 0.588542
epoch 40, train acc 0.687500  test acc 0.649306
epoch 60, train acc 0.750000  test acc 0.694444
epoch 80, train acc 0.765625  test acc 0.704861
epoch 100, train acc 0.828125  test acc 0.724826
epoch 120, train acc 0.828125  test acc 0.756076
epoch 140, train acc 0.859375  test acc 0.769965
epoch 160, train acc 0.859375  test acc 0.777778
epoch 180, train acc 0.890625  test acc 0.801215
epoch 200, train acc 0.921875  test acc 0.816840
epoch 220, train acc 0.906250  test acc 0.818576
epoch 240, train acc 0.906250  test acc 0.821181
epoch 260, train acc 0.906250  test acc 0.821181
epoch 280, train acc 0.906250  test acc 0.829861
epoch 300, train acc 0.921875  test acc 0.838542
epoch 320, train acc 0.921875  test acc 0.843750
epoch 340, train acc 0.937500  test acc 0.848090
epoch 360, train acc 0.953125  test acc 0.856771
epoch 380, train acc 0.953125  test acc 0.855035
epoch 400, train acc 0.921

In [25]:
#Training final model (using all data)
tf.set_random_seed(0)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    count=0
    for j in range(num_epoch):
        for i in range(int(data.shape[0]/batch_size)):
            count+=1
            start_idx=i*batch_size
            end_idx=min((i+1)*batch_size,data.shape[0]-1)
            cur_x=data[start_idx:end_idx,:,:,:]
            cur_y=cls[start_idx:end_idx]
            train_step.run(feed_dict={x_input: cur_x, y_input: cur_y, keep_prob: 0.7})
        if j % 20==0:
            train_accuracy = accuracy.eval(feed_dict={x_input: cur_x, y_input: cur_y, keep_prob: 1})
            print('epoch %d, training accuracy %g' % (j, train_accuracy))

    val_acc=eval_acc(val_data,val_cls,batch_size)
    all_acc=eval_acc(data,cls,batch_size)
    print('validation acc %g' % val_acc)
    print('all data accuracy %g' % all_acc)
    #Save model
    save_path = saver.save(sess, "./tl_classifier.ckpt")
    print('saved')
    

epoch 0, training accuracy 0.40625
epoch 20, training accuracy 0.40625
epoch 40, training accuracy 0.421875
epoch 60, training accuracy 0.4375
epoch 80, training accuracy 0.453125
epoch 100, training accuracy 0.484375
epoch 120, training accuracy 0.546875
epoch 140, training accuracy 0.546875
epoch 160, training accuracy 0.5625
epoch 180, training accuracy 0.578125
epoch 200, training accuracy 0.640625
epoch 220, training accuracy 0.625
epoch 240, training accuracy 0.640625
epoch 260, training accuracy 0.640625
epoch 280, training accuracy 0.65625
epoch 300, training accuracy 0.6875
epoch 320, training accuracy 0.734375
epoch 340, training accuracy 0.703125
epoch 360, training accuracy 0.78125
epoch 380, training accuracy 0.765625
epoch 400, training accuracy 0.796875
epoch 420, training accuracy 0.765625
epoch 440, training accuracy 0.796875
epoch 460, training accuracy 0.859375
epoch 480, training accuracy 0.84375
epoch 500, training accuracy 0.890625
epoch 520, training accuracy 0.8

In [26]:
#Loading test
with tf.Session() as sess:
    saver = tf.train.import_meta_graph("tl_classifier.ckpt.meta")
    saver.restore(sess, "./tl_classifier.ckpt")
    x_input=sess.graph.get_tensor_by_name("x_input:0")
    y_input=sess.graph.get_tensor_by_name("y_input:0")
    prediction=sess.graph.get_tensor_by_name("prediction:0")
    accuracy=sess.graph.get_tensor_by_name("accuracy:0")
    all_acc=eval_acc(data,cls,batch_size)
    print('all data accuracy %g' % all_acc)

INFO:tensorflow:Restoring parameters from ./tl_classifier.ckpt
all data accuracy 0.997253
